<a href="https://colab.research.google.com/github/Eddychege/datascienceprojects/blob/main/MQLs_QI_to_Sign_UPs_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('/content/MQL Q1.csv')

In [5]:
print(df.head())
print(df.info())
print(df.shape)
print(df.columns)

     ` q1aqa1111111111` First Name Second Name                        Email  \
0  NaN        22/1/2025  \nJoseph        Maina    jmwambugu34@gmail.com\n\n   
1  NaN        22/1/2025     Esther     Wanjiru   estyerwanjiru100@gmail.com   
2  NaN        22/1/2025  Fredrick      Micheni  \nfredrickmicheni@gmail.com   
3    1        22/1/2025      Mercy         Jnr        \nmchemjor4@gmail.com   
4    2        22/1/2025      Susan       Hiuhu     \nsusanhiuhu82@gmail.com   

   Phone Number                                Insurer  \
0  2.547818e+11                                    All   
1  2.547178e+11                            APA Medical   
2  2.541145e+11                                    APA   
3  2.547262e+11                      Both PA and Motor   
4  2.547900e+11  ICEA APA Jubilee Britam\nPA Insurance   

                                  Marketing Comments Sales Exec  \
0  Are you a licensed independent insurance agent...     Yvonne   
1  Are you a licensed independent insuranc

In [6]:
df1 = pd.read_csv('/content/1st_transaction_(forecasted_this_month)_listing (2).csv')

In [7]:
print(df1.head())
print(df1.shape)
print(df1.info())
print(df1.columns)

                          Name                       Agency    KYC Mobile  \
0              ERICK  KIPYEGON              ERICK  KIPYEGON     724076521   
1                 BEVERL HONUR                 BEVERL HONUR  254745449685   
2            ANN WAMBUI NGANGA          ANN WAMBUI NG'ANG'A  254112564877   
3  BORN AGAIN INSURANCE AGENCY  BORN AGAIN INSURANCE AGENCY  254723818670   
4         ELVIS AGESA MALIKISI         ELVIS AGESA MALIKISI  254727419101   

                     KYC E-Mail   Status Created Date  Signup Date  \
0        kendagorkips@gmail.com  Sign-up  21-MAR-2025  21-MAR-2025   
1    mbayakhasayabev2@gmail.com  Sign-up  08-JUL-2024  08-JUL-2024   
2   annwambuinganga42@gmail.com  Sign-up  13-JUN-2024  13-JUN-2024   
3  bornagaininsurance@gmail.com  Sign-up  26-JUL-2023  26-JUL-2023   
4          elvisagera@gmail.com  Sign-up  20-JAN-2023  20-JAN-2023   

     Sync Date  Days In Pipeline   Channel    Reffered By  \
0          NaN                66   Pending            N

In [8]:
df2 = pd.read_csv('/content/1st_transaction_(no_forecast_this_month)_listing.csv')

In [9]:
print(df2.head())
print(df2.shape)
print(df2.info())
print(df2.columns)

                              Name                           Agency  \
0                      GEORGE KEYA    DIAMOND MARK INSURANCE AGENCY   
1  MID ALLIANCE INSURANCE AGENCIES  MID ALLIANCE INSURANCE AGENCIES   
2                       LEAH MUNGA                              NaN   
3             GEORGE ODHIAMBO OUMA             GEORGE ODHIAMBO OUMA   
4                     RAHAB MUKAMI                     RAHAB MUKAMI   

     KYC Mobile                      KYC E-Mail   Status Created Date  \
0     722803690  diamondmarkinsurance@yahoo.com  Sign-up  13-JUN-2024   
1  254722804746           gitaujamesk@gmail.com  Sign-up  31-JAN-2025   
2           NaN                             NaN  Sign-up          NaN   
3  254726173612              oushouma@gmail.com  Sign-up  06-FEB-2025   
4  254725813778          rahab.mukami@yahoo.com  Sign-up  04-JUN-2024   

   Signup Date    Sync Date  Days In Pipeline  Channel Reffered By  \
0  13-JUN-2024          NaN               345  Pending         N

In [10]:


# --- Data Cleaning and Standardization ---

# MQL Data (df)
# Rename columns for clarity based on your sample data's structure
# Adjust 'MQL_Internal_ID' and 'MQL_Date' if your actual column names/roles differ
df.rename(columns={
    '`': 'MQL_Internal_ID',
    'q1aqa1111111111`': 'MQL_Date'
}, inplace=True)

# Clean 'Email' column: strip whitespace/newlines and convert to lowercase
df['Email'] = df['Email'].astype(str).str.strip().str.replace('\n', '', regex=False).str.lower()

# Convert 'MQL_Date' to datetime objects (assuming DD/MM/YYYY format)
# 'errors=coerce' will turn unparseable dates into NaT (Not a Time)
df['MQL_Date'] = pd.to_datetime(df['MQL_Date'], dayfirst=True, errors='coerce')

# Drop MQLs with missing essential data (Email or MQL_Date)
df.dropna(subset=['MQL_Date', 'Email'], inplace=True)


# Sign-up Data (df1, df2)
# Concatenate the two sign-up dataframes into one
consolidated_signup_df = pd.concat([df1, df2], ignore_index=True)

# Clean 'KYC E-Mail' column: strip whitespace/newlines and convert to lowercase
consolidated_signup_df['KYC E-Mail'] = consolidated_signup_df['KYC E-Mail'].astype(str).str.strip().str.replace('\n', '', regex=False).str.lower()

# Convert 'Signup Date' to datetime objects (assuming DD-MON-YYYY format)
consolidated_signup_df['Signup Date'] = pd.to_datetime(consolidated_signup_df['Signup Date'], format='%d-%b-%Y', errors='coerce')

# Drop sign-ups with missing essential data (KYC E-Mail or Signup Date)
consolidated_signup_df.dropna(subset=['Signup Date', 'KYC E-Mail'], inplace=True)

# Deduplicate sign-ups: keep the earliest Signup Date for each unique email
consolidated_signup_df.sort_values(by='Signup Date', inplace=True)
consolidated_signup_df.drop_duplicates(subset='KYC E-Mail', keep='first', inplace=True)


# --- Data Merging/Linking ---

# Perform a left merge: Link MQLs to their sign-up records.
# All MQLs are kept, and sign-up info is added if a match exists.
mql_signup_analysis_df = df.merge(
    consolidated_signup_df[['KYC E-Mail', 'Signup Date']],
    left_on='Email',
    right_on='KYC E-Mail',
    how='left'
)

# Create a boolean column to easily identify converted MQLs
mql_signup_analysis_df['Is_Signed_Up'] = mql_signup_analysis_df['Signup Date'].notna()


# --- Conversion Rate Calculation ---

total_mqls = len(df) # The total count of cleaned MQLs
converted_mqls = mql_signup_analysis_df['Is_Signed_Up'].sum() # Count of MQLs with a matched signup date

conversion_rate_signup = (converted_mqls / total_mqls) * 100 if total_mqls > 0 else 0

print(f"--- MQL to Sign-up Conversion Analysis (Overall) ---")
print(f"Total MQLs: {total_mqls}")
print(f"MQLs Converted to Sign-up: {converted_mqls}")
print(f"**Conversion Rate (MQL to Sign-up): {conversion_rate_signup:.2f}%**\n")


# --- Aging Analysis (Time to Convert) ---

# Filter for MQLs that successfully converted and have a logical signup date (not before MQL date)
converted_leads_for_aging = mql_signup_analysis_df[
    mql_signup_analysis_df['Is_Signed_Up'] &
    (mql_signup_analysis_df['Signup Date'] >= mql_signup_analysis_df['MQL_Date'])
].copy() # .copy() to avoid SettingWithCopyWarning

if not converted_leads_for_aging.empty:
    # Calculate the difference in days
    converted_leads_for_aging['Aging_Days'] = (converted_leads_for_aging['Signup Date'] - converted_leads_for_aging['MQL_Date']).dt.days

    average_aging_days = converted_leads_for_aging['Aging_Days'].mean()
    median_aging_days = converted_leads_for_aging['Aging_Days'].median()
    min_aging_days = converted_leads_for_aging['Aging_Days'].min()
    max_aging_days = converted_leads_for_aging['Aging_Days'].max()

    print(f"--- Time to Sign-up (Aging) for Converted MQLs ---")
    print(f"Average: {average_aging_days:.2f} days")
    print(f"Median: {median_aging_days:.2f} days")
    print(f"Minimum: {min_aging_days} days")
    print(f"Maximum: {max_aging_days} days")
else:
    print("No converted MQLs found with a logical signup date to calculate aging.")

--- MQL to Sign-up Conversion Analysis (Overall) ---
Total MQLs: 115
MQLs Converted to Sign-up: 5
**Conversion Rate (MQL to Sign-up): 4.35%**

--- Time to Sign-up (Aging) for Converted MQLs ---
Average: 27.75 days
Median: 28.00 days
Minimum: 1 days
Maximum: 54 days


In [11]:
# Assuming the previous code snippet has been executed and 'converted_leads_for_aging' DataFrame exists

# Display the relevant columns for each converted MQL
if not converted_leads_for_aging.empty:
    print("\n--- Details of Each Converted MQL (MQL to Sign-up) ---")
    # Select and display the specific columns requested
    print(converted_leads_for_aging[['Email', 'MQL_Date', 'Signup Date', 'Aging_Days']].to_string(index=False))
else:
    print("\nNo MQLs converted to Sign-up (with logical dates) to display individual details.")



--- Details of Each Converted MQL (MQL to Sign-up) ---
                 Email   MQL_Date Signup Date  Aging_Days
kendagorkips@gmail.com 2025-01-26  2025-03-21          54
    oushouma@gmail.com 2025-02-02  2025-02-06           4
    jwmugo@outlook.com 2025-02-16  2025-04-09          52
kendagorkips@gmail.com 2025-03-20  2025-03-21           1


In [12]:
# Assuming the previous code snippet has been executed and 'converted_leads_for_aging' DataFrame exists

# Display the relevant columns for each converted MQL, including names
if not converted_leads_for_aging.empty:
    print("\n--- Details of Each Converted MQL (MQL to Sign-up) with Names ---")
    # Select and display the specific columns, now including 'First Name' and 'Second Name'
    print(converted_leads_for_aging[['First Name', 'Second Name', 'Email', 'MQL_Date', 'Signup Date', 'Aging_Days']].to_string(index=False))
else:
    print("\nNo MQLs converted to Sign-up (with logical dates) to display individual details.")


--- Details of Each Converted MQL (MQL to Sign-up) with Names ---
First Name Second Name                  Email   MQL_Date Signup Date  Aging_Days
     Erick    Kipyegon kendagorkips@gmail.com 2025-01-26  2025-03-21          54
    George    Odhiambo     oushouma@gmail.com 2025-02-02  2025-02-06           4
      Jane        Mugo     jwmugo@outlook.com 2025-02-16  2025-04-09          52
     Erick    Kipyegon kendagorkips@gmail.com 2025-03-20  2025-03-21           1
